# Assignment #4: Extracting syntactic groups using machine-learning techniques
Author: Pierre Nugues

In this assignment, you will create a system to extract syntactic groups from a text. You will apply it to the CoNLL 2000 dataset. In addition, you will try to link a few extracted named entities to real things using wikipedia.

## Objectives

The objectives of this assignment are to:
* Write a program to detect partial syntactic structures
* Extract named entities and link them to real things using Wikipedia
* Understand the principles of supervised machine learning techniques applied to language processing
* Use a popular machine learning toolkit: scikit-learn
* Write a short report of 2 to 3 pages on the assignment

#### Reading the files

Read the functions below to load the datasets. They store the corpus in a list of sentences. Each sentence is a list of rows, where each row is a dictionary.

In [176]:
def read_sentences(file):
    """
    Creates a list of sentences from the corpus
    Each sentence is a string
    :param file:
    :return:
    """
    f = open(file).read().strip()
    sentences = f.split('\n\n')
    return sentences

In [177]:
def split_rows(sentences, column_names):
    """
    Creates a list of sentence where each sentence is a list of lines
    Each line is a dictionary of columns
    :param sentences:
    :param column_names:
    :return:
    """
    new_sentences = []
    for sentence in sentences:
        rows = sentence.split('\n')
        sentence = [dict(zip(column_names, row.split())) for row in rows]
        new_sentences.append(sentence)
    return new_sentences

### Loading dictionaries

The CoNLL 2000 files have three columns

In [178]:
column_names = ['form', 'pos', 'chunk']

We load the corpus

In [ ]:
train_sentences = read_sentences(train_file)
train_corpus = split_rows(train_sentences, column_names)
train_corpus[:2]

"""
[[{'form': 'Confidence', 'pos': 'NN', 'chunk': 'B-NP'},
  {'form': 'in', 'pos': 'IN', 'chunk': 'B-PP'},
  {'form': 'the', 'pos': 'DT', 'chunk': 'B-NP'},
  {'form': 'pound', 'pos': 'NN', 'chunk': 'I-NP'},
    ...
"""

## Baseline chunker

Most statistical algorithms for language processing start with a so-called baseline. The baseline performance corresponds to the application of a minimal technique that is used to assess the difficulty of a task and for comparison with further programs.

### Auxiliary functions

A function to count the parts of speech

In [180]:
def count_pos(corpus):
    """
    Computes the part-of-speech distribution
    in a CoNLL 2000 file
    :param corpus:
    :return:
    """
    pos_cnt = {}
    for sentence in corpus:
        for row in sentence:
            if row['pos'] in pos_cnt:
                pos_cnt[row['pos']] += 1
            else:
                pos_cnt[row['pos']] = 1
    return pos_cnt

We first collect all the parts of speech and we count them.

In [ ]:
pos_cnt = count_pos(train_corpus)
pos_cnt
""" Output:
{'NN': 30147,
 'IN': 22764,
 'NNS': 13619,
 'NNP': 19884,
 ...

"""

### Chunk distribution

You will compute the chunk distribution for each part of speech. You will use the training file to derive the distribution and you will store the results in a dictionary. Below, you have an excerpt of the expected results:
```
{'JJR':
{'I-ADVP': 17, 'I-ADJP': 45, 'I-NP': 204, 'B-ADVP': 63,
'B-PP': 2, 'B-ADJP': 111, 'B-NP': 382, 'B-VP': 2,
'I-VP': 11, 'O': 16},
...
```

In [182]:
from collections import defaultdict

def chunk_distribution(corpus):
    chunk_dist = defaultdict(dict)
    for sentence in corpus:
        for row in sentence:
            _, pos, chunk = row.values()
            if chunk in chunk_dist[pos]:
                chunk_dist[pos][chunk] += 1
            else:
                chunk_dist[pos][chunk] = 1
    return chunk_dist

chunk_dist = chunk_distribution(train_corpus)

In [ ]:
chunk_dist['NN']
"""
{'B-NP': 5160,
 'I-NP': 24456,
 'B-VP': 257,
 'B-ADJP': 44,
 'B-ADVP': 38,
 '...
 """

### Selecting the POS-chunk associations

For each part of speech, select the best association. In the example above, you will have (NN, I-NP) as it is the most frequent. You will store the results in a dictionary that you will call `pos_chunk`

In [184]:
# Sort all dicts in chunk dist

pos_chunk = {}

for pos in chunk_dist.keys():
    _dict = chunk_dist[pos]
    max_key = max(_dict, key=_dict.get)
    pos_chunk[pos] = max_key

In [185]:
pos_chunk['NN']

'I-NP'

### Prediction

Using the resulting associations, apply your chunker to the test file. You will write a `predict(model, corpus)` function, where `model` will be your associations and `corpus`, the test corpus. You will format the test corpus as a dictionary, where you will add a `pchunk` key for each row with a value that will correspond to the predicted chunk.

In [186]:
def predict(model, corpus):
    predicted_corpus = corpus.copy()
    for sent_idx, sentence in enumerate(corpus):
        for row_idx, row in enumerate(sentence):
            _, pos, _ = row.values()
            predicted_corpus[sent_idx][row_idx]['pchunk'] = model[pos]
    return predicted_corpus

We load the test corpus

In [ ]:
test_sentences = read_sentences(test_file)
test_corpus = split_rows(test_sentences, column_names)
test_corpus[:1]
"""
[[{'form': 'Rockwell', 'pos': 'NNP', 'chunk': 'B-NP'},
  {'form': 'International', 'pos': 'NNP', 'chunk': 'I-NP'},
  {'form': 'Corp.', 'pos': 'NNP', 'chunk': 'I-NP'},
  {'form': "'s", 'pos': 'POS', 'chunk': 'B-NP'},
"""

We predict the groups. You should have added a `pchunk` key

In [ ]:
predicted_test_corpus = predict(pos_chunk, test_corpus)
predicted_test_corpus[:1]
"""
[[{'form': 'Rockwell', 'pos': 'NNP', 'chunk': 'B-NP', 'pchunk': 'I-NP'},
  {'form': 'International', 'pos': 'NNP', 'chunk': 'I-NP', 'pchunk': 'I-NP'},
  {'form': 'Corp.', 'pos': 'NNP', 'chunk': 'I-NP', 'pchunk': 'I-NP'},
  {'form': "'s", 'pos': 'POS', 'chunk': 'B-NP', 'pchunk': 'B-NP'},
  """

### Accuracy

We can evaluate the performance of the baseline with the tag accuracy: the percentage of words that receive the correct tag.

In [189]:
def eval(predicted):
    """
    Evaluates the predicted chunk accuracy
    :param predicted:
    :return:
    """
    word_cnt = 0
    correct = 0
    for sentence in predicted:
        for row in sentence:
            word_cnt += 1
            if row['chunk'] == row['pchunk']:
                correct += 1
    return correct / word_cnt

In [190]:
accuracy = eval(predicted_test_corpus)
accuracy

0.7729066846782194

### The CoNLL evaluation

The accuracy is very misleading as it is biased by the most frequent tags. It is not a good way to evaluate chunking. Instead, CoNLL computes the F1 score of all the chunks with a specific evaluation script.

#### Saving the corpus

To use the CoNLL evaluation script, you will store your results in an output file that has four columns. The three first columns will be the input columns from the test file: 
* word, 
* part of speech, and 
* gold-standard chunk. 

You will append the predicted chunk as the 4th column. Your output file should look like the excerpt below:
```
Rockwell NNP B-NP I-NP
International NNP I-NP I-NP
Corp. NNP I-NP I-NP
's POS B-NP B-NP
Tulsa NNP I-NP I-NP
unit NN I-NP I-NP
said VBD B-VP B-VP
it PRP B-NP B-NP
```
The separator is the space.

You will use a `save_results(output_dict, keys, output_file)` function, where the keys will be `['form', 'pos', 'chunk', 'pchunk']`

In [191]:
keys = ['form', 'pos', 'chunk', 'pchunk']

In [192]:
def save_results(output_dict, keys, output_file):
    f_out = open(output_file, 'w')
    # We write the word (form), part of speech (pos),
    # gold-standard chunk (chunk), and predicted chunk (pchunk)
    for sentence in output_dict:
        for row in sentence:
            for key in keys:
                f_out.write(row[key] + ' ')
            f_out.write('\n')
        f_out.write('\n')
    f_out.close()
    return

In [193]:
save_results(predicted_test_corpus, keys, 'out')

The CoNLL 2000 evaluation script will use these two last columns, chunk and predicted chunk, to compute the performance.

## Using Machine Learning: A first ML program

In this exercise, you will apply and explore a machine-learning program.

The program that won the CoNLL 2000 shared task (Kudoh and Matsumoto, 2000) used a window of five words around the chunk tag to identify, $c_i$. They built a feature vector consisting of:
1. The values of the five words in this window: $w_{i-2}, w_{i-1}, w_{i}, w_{i+1}, w_{i+2}$
2. The values of the five parts of speech in this window: $t_{i-2}, t_{i-1}, t_{i}, t_{i+1}, t_{i+2}$
3. The values of the two previous chunk tags in the first part of the window: $c_{i-2}, c_{i-1}$

The two last parameters (3.) are said to be dynamic because the program computes them at run-time. Read [Kudoh and Matsumoto's paper](http://www.clips.uantwerpen.be/conll2000/pdf/14244kud.pdf) and the [Yamcha](http://www.chasen.org/~taku/software/yamcha/) software site.

You will start with a given code that uses the two first sets of features (1. and 2.) and add yourself the last one (3.) to improve the performance of your chunker. Kudoh and Matsumoto trained a classifier based on support vector machines. You will use logistic regression.

### Feature extraction

#### Functions

A first function to extract features from one sentence

In [197]:
def extract_features_sent_static(sentence, w_size, feature_names):
    """
    Extract the features from one sentence
    returns X and y, where X is a list of dictionaries and
    y is a list of symbols
    :param sentence: string containing the CoNLL structure of a sentence
    :param w_size:
    :return:
    """

    # We pad the sentence to extract the context window more easily
    start = [{'form': 'BOS', 'pos': 'BOS', 'chunk': 'BOS'}]
    end = [{'form': 'EOS', 'pos': 'EOS', 'chunk': 'EOS'}]
    start *= w_size
    end *= w_size
    padded_sentence = start + sentence
    padded_sentence += end

    # We extract the features and the classes
    # X contains is a list of features, where each feature vector is a dictionary
    # y is the list of classes
    X = list()
    y = list()
    for i in range(len(padded_sentence) - 2 * w_size):
        # x is a row of X
        x = list()
        # The words in lower case
        for j in range(2 * w_size + 1):
            x.append(padded_sentence[i + j]['form'].lower())
        # The POS
        for j in range(2 * w_size + 1):
            x.append(padded_sentence[i + j]['pos'])
        # The chunks (Up to the word)
        """
        for j in range(w_size):
            feature_line.append(padded_sentence[i + j]['chunk'])
        """
        # We represent the feature vector as a dictionary
        X.append(dict(zip(feature_names, x)))
        # The classes are stored in a list
        y.append(padded_sentence[i + w_size]['chunk'])
    return X, y

And from all the sentences

In [198]:
def extract_features_static(sentences, w_size, feature_names):
    """
    Builds X matrix and y vector
    X is a list of dictionaries and y is a list
    :param sentences:
    :param w_size:
    :return:
    """
    X_l = []
    y_l = []
    for sentence in sentences:
        X, y = extract_features_sent_static(sentence, w_size, feature_names)
        X_l.extend(X)
        y_l.extend(y)
    return X_l, y_l

#### Applying the feature extraction

The size of the window and the names of the features

In [199]:
w_size = 2  # The size of the context window to the left and right of the word
feature_names = ['word_n2', 'word_n1', 'word', 'word_p1', 'word_p2',
                 'pos_n2', 'pos_n1', 'pos', 'pos_p1', 'pos_p2']

We read the corpus and format it as a dictionary

In [200]:
train_sentences = read_sentences(train_file)
train_corpus = split_rows(train_sentences, column_names)

In [ ]:
X_dict, y = extract_features_static(train_corpus, w_size, feature_names)
X_dict[:2]

In [203]:
y[:2]

['B-NP', 'B-PP']

### Feature encoding

In [51]:
# Vectorize the feature matrix and carry out a one-hot encoding
vec = DictVectorizer(sparse=True)
X = vec.fit_transform(X_dict)

### Training the model

In [ ]:
classifier = linear_model.LogisticRegression()
model = classifier.fit(X, y)
model

### Predicting the test set

We read the sentences and create a dictionary

In [ ]:
test_sentences = read_sentences(test_file)
test_corpus = split_rows(test_sentences, column_names)
test_corpus[:2]

We extract the features

In [ ]:
X_test_dict, y_test = extract_features_static(test_corpus, w_size, feature_names)
X_test_dict[:2]
"""
[{'word_n2': 'bos',
  'word_n1': 'bos',
  'word': 'rockwell',
  'word_p1': 'international',
  'word_p2': 'corp.',
  'pos_n2': 'BOS',
  'pos_n1': 'BOS',
  'pos': 'NNP',
  'pos_p1': 'NNP',
  'pos_p2': 'NNP'},
  ...
"""

In [55]:
y_test[:2]

['B-NP', 'I-NP']

We vectorize the features

In [56]:
X_test = vec.transform(X_test_dict)  # Possible to add: .toarray()

And we predict the test set

In [57]:
y_test_predicted = classifier.predict(X_test)
y_test_predicted[:2]

array(['B-NP', 'I-NP'], dtype='<U7')

We now add the predicted chunks to the sentences

In [58]:
inx = 0
for sentence in test_corpus:
    for word in sentence:
        word['pchunk'] = y_test_predicted[inx]
        inx += 1

The index sould be equal to the length of the prediction

### Question on the ML program

1. What is the feature vector that corresponds to the <tt>ml_chunker.py</tt> program? Is it the same Kudoh
    and Matsumoto used in their experiment?
2. What is the performance of the chunker?
3. Remove the lexical features (the words) from the feature vector and measure the performance. You should
    observe a decrease.
4. What is the classifier used in the program? 
5. As an optional task, you may try two other classifiers from sklearn and measure their performance: decision trees, perceptron, support vector machines, etc. Be aware that support vector machines take a long time to train: up to one hour.

## Using Machine Learning: Adding all the features from Kudoh and Matsumoto

Complement the feature vector used in the previous section with the two dynamic features, $c_{i-2}, c_{i-1}$, and train a new model. You will need to write a new `extract_features_sent_dyn` and `predict` functions. 
In his experiments, your teacher obtained a F1 score of 92.65 with logistic regression and the default parameters from sklearn, i.e. `linear_model.LogisticRegression()`;

**A frequent mistake in the labs** is to use the gold-standard chunks from the test set. Be aware that  when you predict the test set, you do not know the dynamic features in advance and you must  not use the ones from the test file. You will use the two previous chunk tags that you have predicted.

You need to reach a global F1 score of 92 to pass this laboratory.

In [204]:
def extract_features_sent_dyn(sentence, w_size, feature_names):
    # We pad the sentence to extract the context window more easily
    start = [{'form': 'BOS', 'pos': 'BOS', 'chunk': 'BOS'}]
    end = [{'form': 'EOS', 'pos': 'EOS', 'chunk': 'EOS'}]
    start *= w_size
    end *= w_size
    padded_sentence = start + sentence
    padded_sentence += end

    # We extract the features and the classes
    # X contains is a list of features, where each feature vector is a dictionary
    # y is the list of classes
    X = list()
    y = list()
    for i in range(len(padded_sentence) - 2 * w_size):
        # x is a row of X
        x = list()
        # The words in lower case
        for j in range(2 * w_size + 1):
            x.append(padded_sentence[i + j]['form'].lower())
        # The POS
        for j in range(2 * w_size + 1):
            x.append(padded_sentence[i + j]['pos'])
        # The chunks (Up to the word)
        for j in range(w_size):
            x.append(padded_sentence[i + j]['chunk'])
        
        # We represent the feature vector as a dictionary
        X.append(dict(zip(feature_names, x)))
        # The classes are stored in a list
        y.append(padded_sentence[i + w_size]['chunk'])
    return X, y

In [205]:
def extract_features_dyn(sentences, w_size, feature_names):
    """
    Builds X matrix and y vector
    X is a list of dictionaries and y is a list
    :param sentences:
    :param w_size:
    :return:
    """
    X_l = []
    y_l = []
    for sentence in sentences:
        X, y = extract_features_sent_dyn(sentence, w_size, feature_names)
        X_l.extend(X)
        y_l.extend(y)
    return X_l, y_l

In [206]:
w_size = 2
feature_names_dyn = ['word_n2', 'word_n1', 'word', 'word_p1', 'word_p2',
                     'pos_n2', 'pos_n1', 'pos', 'pos_p1', 'pos_p2', 
                     'chunk_n2', 'chunk_n1']

In [ ]:
train_sentences = read_sentences(train_file)
train_corpus = split_rows(train_sentences, column_names)
train_corpus[:2]

In [208]:
X_dict, y = extract_features_dyn(train_corpus, w_size, feature_names_dyn)

You will now vectorize the training set

In [210]:
vec = DictVectorizer(sparse=True)
X_train = vec.fit_transform(X_dict)

And fit the model

In [ ]:
classifier2 = linear_model.LogisticRegression()
model2 = classifier2.fit(X_train, y)

### Prediction

You will finally predict the test set. We load the corpus again.

In [ ]:
test_sentences = read_sentences(test_file)
test_corpus = split_rows(test_sentences, column_names)
test_corpus[:2]

Let us extract the static features from one sentence

In [ ]:
X_test_dict, y_test = extract_features_static([test_corpus[0]], w_size, feature_names)
X_test_dict[:2]

This $\mathbf{X}\_{\textrm{dict}}$ is incomplete. For the prediction, we need to reinject dynamically the two previously predicted tags to have the full feature vector. Write this code here. 

This part is probably the most difficult of the lab. You may want to write it first for one sentence, and then for the test corpus. The prediction will take a longer time and you may want to include a progress bar with this snippet: 
```
from tqdm import tqdm
for test_sentence in tqdm(test_corpus):
```

In [215]:
y_test[:2]

['B-NP', 'I-NP']

In [216]:
y_test_predicted_dyn = []

In [ ]:
from tqdm import tqdm
for test_sentence in tqdm(test_corpus):
    chunk_n1, chunk_n2 = 'BOS', 'BOS'
    X_test_dict, y_test = extract_features_static([test_sentence], w_size, feature_names)
    for i in range(len(X_test_dict)):
        X_test_dict[i]['chunk_n1'] = chunk_n1
        X_test_dict[i]['chunk_n2'] = chunk_n2
        X_test = vec.transform(X_test_dict[i])
        y_pred = classifier2.predict(X_test)
        
        y_test_predicted_dyn.extend(y_pred)
        chunk_n2 = chunk_n1
        chunk_n1 = y_pred[0]


In [219]:
y_test_predicted_dyn[:3]

['B-NP', 'I-NP', 'I-NP']

In [220]:
inx = 0
for sentence in test_corpus:
    for word in sentence:
        word['pchunk'] = y_test_predicted_dyn[inx]
        inx += 1

In [221]:
save_results(test_corpus, keys, 'out')

#### Evaluation

In [222]:
lines = open('out').read().splitlines()
res = conlleval.evaluate(lines)
improved_ml_score = res['overall']['chunks']['evals']['f1']
improved_ml_score

0.9265266775640221

## Collecting the entities

You will now collect all the named entities from the training set, defined as NP chunks and starting with a `NNP` (proper noun) or a `NNPS` (proper noun, plural) tag. As an example, in the first sentence of `train_corpus`, you will extract `('September', )` and `('July', 'and', 'August')`. You will set all the tuples in a set that you will call `ne_set`.

You can write a two-pass procedure. For each sentence of the corpus:
1. In the first pass, you will collect the start indices of the noun groups which are also proper nouns. For the first sentence, it will result in the list `[16, 30]`;
2. In the second pass, you will collect the segments, starting at each index. For the first sentence, it will result in the tuples `('September',)`and `('July', 'and', 'August')`

Should you have a better solution, please use it.

In [224]:
ne_set = set()
for sentence in train_corpus:
    i = 0
    while i < len(sentence):
        idxs = []
        # If word at index i in sentence is B-NP:
        if sentence[i]['pos'] in ['NNP', 'NNPS'] and sentence[i]['chunk'] == 'B-NP':
            idxs.append(i)
            i += 1
            while i < len(sentence) and sentence[i]['chunk'] == 'I-NP':
                idxs.append(i)
                i += 1
            # Add tuple of the forms at the extracted word indexes to ne_set
            _set = []
            for idx in idxs:
                _set.append(sentence[idx]['form'])
            ne_set.add(tuple(_set))
        else:
            i += 1

In [226]:
list(ne_set)[:10]

[('Lotus', 'Development'),
 ('Chandler',),
 ('Mr.', 'Lowell'),
 ('Economic', 'Advisers'),
 ('R.I',),
 ('Moody', "'s"),
 ('Minnesota', 'Mining', '&', 'Manufacturing', 'Co.'),
 ('RJR', 'Holdings', 'Capital', 'Corp.'),
 ('Lewis', 'Galoob', 'Toys'),
 ('September', '1983')]

### Creating a small set

To run the subsequent experiments faster, you will limit the dataset to the entities starting with letter _K_. I chose this letter, because it corresponded to one of the smallest sets. You will call the resulting set: `ne_small_set`. Feel free to use the full set after you have completed this assignment.

In [230]:
ne_small_set = set(entity for entity in ne_set if entity[0][0].lower() == 'k')

## Resolving the entities

You will now implement a simple method to find the named entities from the previous exercise in Wikipedia and Wikidata.

### A function to lookup entities

Read the function below and try to understand what it means. You will describe it in your report.

In [248]:
def wikipedia_lookup(ner, base_url='https://en.wikipedia.org/wiki/'):
    try:
        url_en = base_url + ' '.join(ner)
        html_doc = requests.get(url_en).text
        parse_tree = bs4.BeautifulSoup(html_doc, 'html.parser')
        entity_id = parse_tree.find("a", {"accesskey": "g"})['href']
        head_id, entity_id = os.path.split(entity_id)
        return entity_id
    except:
        pass
        # print('Not found in: ', base_url)
    entity_id = 'UNK'
    return entity_id

Write a function to run the lookup and keep the resolved entities (only the resolved entities). You will call it `ne_ids_en`

In [249]:
ne_ids_en = []
for entity in ne_small_set:
    entity_id = wikipedia_lookup(entity)
    if entity_id != 'UNK':
        ne_ids_en.append((entity, entity_id))

Sometimes, entities need a confirmation. You will apply the resolution with the Swedish wikipedia.

In [251]:
ne_ids_sv = []
for entity in ne_small_set:
    entity_id = wikipedia_lookup(entity, base_url='https://sv.wikipedia.org/wiki/')
    if entity_id != 'UNK':
        ne_ids_sv.append((entity, entity_id))

You will compute the intersection of the two sets. You will assign it to a list that you will sort and that you will call: `confirmed_ne_en_sv`.

In [253]:
confirmed_ne_en_sv = list(set(ne_ids_en).intersection(set(ne_ids_sv)))
confirmed_ne_en_sv = sorted(confirmed_ne_en_sv)

The first items in your list should look like:
```
[(('KIM',), 'Q224736'),
 (('KLM', 'Royal', 'Dutch', 'Airlines'), 'Q181912'),
 ...
]
```